In [3]:
!pip install scikit-image

     |████████████████████████████████| 26.5 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 811 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 264 kB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 1.1 MB/s eta 0:00:01


In [4]:
'''
Which functions do I have to make?? 

run inference per image 
according to its scale 
then save the images into folders if there is 
also I have to make golden samples 

'''

import numpy as np
import slidingwindow as sw
import sys
import os

from mmdet.apis import init_detector, inference_detector
from mmcv.runner import load_checkpoint



# import shm_tools here
sys.path.append('..')
from shm_tools.shm_utils import imread, imwrite

In [5]:
## download checkpoint files from the rtx server 

config = './project_work_dirs/express_cor/configs/cascade_mask_rcnn_x101_64x4d_fpn_carafe_1x_road_crack_200318.py'
checkpoint = './project_work_dirs/express_cor/work_dirs/cascade_mask_rcnn_x101_64x4d_fpn_carafe_1x_road_crack_200318/epoch_30.pth'

device = 'cuda:0'
model = init_detector(config, checkpoint, device=device)


In [41]:
filename_base[0]

'DSC00167'

In [ ]:
imsave_folder = 'data/Gongju_bridge_image_using_drone/#_area_with_crack_detection'

for root, dirs, files in os.walk('data/Gongju_bridge_image_using_drone'):
    for file in files:
        if file.endswith(".JPG") or file.endswith(".jpg") :
#             print(os.path.join(root, file))
            img = imread(os.path.join(root, file))
            windows = sw.generate(img, sw.DimOrder.HeightWidthChannel, 1024+256, 0 )
            img_num = 0
        
            for window in windows: # Can a multithreading be used here?? 
                img_num += 1 

                img_subset = img[window.indices()]
                results = inference_detector(model, img_subset)
                bbox_results, seg_results = results
                for bbox in bbox_results[0]: 
                    if bbox[-1] > 0.5:
                        filename_base, _ = os.path.splitext(file)
                        img_num_str = str(img_num)
                        img_num_str = img_num_str.zfill(3)
                        imsave_filename = filename_base + '_' + img_num_str + '.jpeg'
                        imwrite(os.path.join(imsave_folder, imsave_filename), img_subset )
                        break
                
            
    

In [27]:
for bbox in bbox_results[0]: 
    print(bbox[-1])
    if bbox[-1] > 0.1:
        imwrite(img_subset, )

0.086606674
0.06438841
0.06340289
0.058315504
0.08071749
0.11163888


In [ ]:

img_folder = '/home/soojin/mmdetection_carafe/demo/exdemo_200220/아스팔트/'
img_path_list = glob.glob(os.path.join(img_folder, '*.jpg')) + glob.glob(os.path.join(img_folder, '*.JPG'))

for img_path in img_path_list : 

    img = imread(img_path)
    img_result = img.copy()
    score_thr = 0.1
    # Generate the set of windows, with a 256-pixel max window size and 50% overlap

    windows = sw.generate(img, sw.DimOrder.HeightWidthChannel, 256+512, 0.5)
    mask_output = np.zeros((img.shape[0], img.shape[1]), dtype=np.bool)

    for window in windows: # Can a multithreading be used here?? 

        img_subset = img[window.indices()]
        results = inference_detector(model, img_subset)